In [3]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize, sent_tokenize

In [44]:
from gensim import models
import numpy as np

In [10]:
import gensim
from gensim import corpora
from pprint import pprint

In [4]:
df.head() 

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [81]:
df_chunk = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv", chunksize=100000)

In [82]:
content_en_list = []  # append the content from each chunk df here 

In [83]:
%%time
# Each chunk is in df format
for chunk in df_chunk:  
    # perform data filtering 
    chunk_filter = chunk.drop(['Unnamed: 0'], axis=1)
    chunk_filter = chunk.content_en
    print(type(chunk_filter))
    
    # Once the data filtering is done, append the chunk to list
    content_en_list.append(chunk_filter)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
Wall time: 28.4 s


In [84]:
len(tokenized_content)

0

In [85]:
%%time
df_concat = pd.concat(content_en_list)

Wall time: 12 ms


In [86]:
df_concat[0]

'<p>Mr. Speaker, I rise on a point of order. I believe you would find unanimous consent following consultation among all parties to further defer the vote just deferred until Monday to next Tuesday at the end of government orders.</p>'

getting the english statements

In [87]:
def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    words = word_tokenize(text)
    return words

In [88]:
from nltk.corpus import stopwords

In [89]:
stop_words = set(stopwords.words('english'))

In [ ]:
tokens = df.tokenized_content[2]

In [ ]:
def remove_stopwords(word_tokens):
    words = [w for w in word_tokens if not w in stop_words]
    return words

In [ ]:
remove_stopwords(word_tokens=tokens)

In [ ]:
tokens

In [11]:
documents = [ 
             "Saudi journalist Jamal Khashoggi's death was the result of an", 
             "interrogation that went wrong, one that was intended to lead", 
             "to his abduction from Turkey, according to two sources."]

documents_2 = ["One source says the report will likely conclude that", 
                "the operation was carried out without clearance and", 
                "transparency and that those involved will be held", 
                "responsible. One of the sources acknowledged that the", 
                "report is still being prepared and cautioned that", 
                "things could change."]

In [15]:
texts = [[text for text in doc.split()] for doc in documents]

In [16]:
dictionary = corpora.Dictionary(texts)

In [17]:
print(dictionary)

Dictionary(33 unique tokens: ['Saudis', 'The', 'a', 'acknowledge', 'are']...)


texts is a list of tokens

In [36]:
statements = df_concat[:1000]

In [37]:
%%time
tokenized_statements = statements.apply(tokenize_column)

Wall time: 7.16 s


In [38]:
%%time
dictionary = corpora.Dictionary(tokenized_statements)

Wall time: 547 ms


In [58]:
%%time
mydict = corpora.Dictionary(statements.values)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [64]:
corpus = [mydict.doc2bow(line) for line in statements.values]

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [52]:
tfidf = models.TfidfModel(corpus, smartirs='ntc')

In [55]:
type(statements)

pandas.core.series.Series

In [57]:
type(statements.values)

numpy.ndarray

In [61]:
s = statements.values

In [66]:
s[0]

"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a', 'point', 'of', 'order', '.', 'i', 'believe', 'you', 'would', 'find', 'unanimous', 'consent', 'following', 'consultation', 'among', 'all', 'parties', 'to', 'further', 'defer', 'the', 'vote', 'just', 'deferred', 'until', 'monday', 'to', 'next', 'tuesday', 'at', 'the', 'end', 'of', 'government', 'orders', '.']"

In [72]:
tokenize_column(df_concat[0])

['[',
 "'mr",
 '.',
 "'",
 ',',
 "'speaker",
 "'",
 ',',
 "'",
 ',',
 "'",
 ',',
 "'",
 'i',
 "'",
 ',',
 "'rise",
 "'",
 ',',
 "'on",
 "'",
 ',',
 "'",
 'a',
 "'",
 ',',
 "'point",
 "'",
 ',',
 "'of",
 "'",
 ',',
 "'order",
 "'",
 ',',
 "'",
 '.',
 "'",
 ',',
 "'",
 'i',
 "'",
 ',',
 "'believe",
 "'",
 ',',
 "'you",
 "'",
 ',',
 "'would",
 "'",
 ',',
 "'find",
 "'",
 ',',
 "'unanimous",
 "'",
 ',',
 "'consent",
 "'",
 ',',
 "'following",
 "'",
 ',',
 "'consultation",
 "'",
 ',',
 "'among",
 "'",
 ',',
 "'all",
 "'",
 ',',
 "'parties",
 "'",
 ',',
 "'to",
 "'",
 ',',
 "'further",
 "'",
 ',',
 "'defer",
 "'",
 ',',
 "'the",
 "'",
 ',',
 "'vote",
 "'",
 ',',
 "'just",
 "'",
 ',',
 "'deferred",
 "'",
 ',',
 "'until",
 "'",
 ',',
 "'monday",
 "'",
 ',',
 "'to",
 "'",
 ',',
 "'next",
 "'",
 ',',
 "'tuesday",
 "'",
 ',',
 "'at",
 "'",
 ',',
 "'the",
 "'",
 ',',
 "'end",
 "'",
 ',',
 "'of",
 "'",
 ',',
 "'government",
 "'",
 ',',
 "'orders",
 "'",
 ',',
 "'",
 '.',
 "'",
 ']']